# mount google drive and load utility functions

In [1]:
!pip install mlflow

In [2]:
import pathlib
import os

#mount google drive
from google.colab import drive
DRIVE_PATH = pathlib.Path("/content/drive")
drive.mount(str(DRIVE_PATH))

#import utils
PROJ = "MyDrive/github/dogclassifier"
PROJECT_PATH = DRIVE_PATH.joinpath(PROJ)
UTILS_PATH = PROJECT_PATH.joinpath("utils")

os.chdir(UTILS_PATH)
import utils
# from graphs_keras import KPlot
from train_keras import KTrain
from data_keras import KData

os.chdir(PROJECT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# get dataset as train/test split, fixed preprocessing

In [3]:
kdata_obj = KData()
#load data with defaults
train_set, test_set, class_names = kdata_obj.load_data()
#transform into batches
train_set = kdata_obj.get_train_batches(train_set)
test_set = kdata_obj.get_test_batches(test_set)

# mlflow setup


In [4]:
import mlflow.tensorflow
# mlflow.tensorflow.autolog(every_n_iter=2)

#tracking directory
# uri = 
# mlflow.set_tracking_uri(uri)

import tempfile


# ResNet50_v2 model

In [9]:
from tensorflow.keras import layers
import tensorflow as tf

In [10]:
#resnet50v2 base model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess
resnet_base_model = ResNet50V2(include_top=False,
                               weights='imagenet',
                               pooling='avg' #apply global average pooling to output of last conv block
                               )
resnet_base_model.trainable = False #freeze base model layers


#create model
input_ = layers.Input(shape=(224, 224, 3), name='input_layer')
x = layers.Lambda(resnet_preprocess)(input_)
x = resnet_base_model(x, training=False)
output_ = layers.Dense(len(class_names), activation='softmax', name='output_layer')(x)
resnet50v2_feature_model = tf.keras.Model(inputs=[input_], outputs=[output_])
resnet50v2_feature_model.summary()


94683136/94668760 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 output_layer (Dense)        (None, 120)               245880    
                                                                 
Total params: 23,810,680
Trainable params: 245,880
Non-trainable params: 23,564,800
_________________________________________________________________


In [11]:
ktrain_obj = KTrain()
ktrain_obj.train_model(resnet50v2_feature_model, train_set, test_set, class_names, epochs=10, expt_name='resnet50_v2', run_name='baseline')

Compiling model...
Writing logs to /tmp/tmpd7h9lkdo
Epoch 1/10
375/375 [==============================] - 32s 72ms/step - loss: 1.4830 - accuracy: 0.6094 - val_loss: 1.0120 - val_accuracy: 0.7086
Epoch 2/10
375/375 [==============================] - 27s 68ms/step - loss: 0.5601 - accuracy: 0.8257 - val_loss: 1.0305 - val_accuracy: 0.7211
Epoch 3/10
375/375 [==============================] - 28s 70ms/step - loss: 0.3315 - accuracy: 0.9028 - val_loss: 1.0425 - val_accuracy: 0.7320
Epoch 4/10
375/375 [==============================] - 27s 67ms/step - loss: 0.2123 - accuracy: 0.9450 - val_loss: 1.0824 - val_accuracy: 0.7266
Epoch 5/10
375/375 [==============================] - 27s 68ms/step - loss: 0.1425 - accuracy: 0.9698 - val_loss: 1.0869 - val_accuracy: 0.7102
Epoch 6/10
375/375 [==============================] - 27s 68ms/step - loss: 0.0996 - accuracy: 0.9818 - val_loss: 1.1170 - val_accuracy: 0.7258
Epoch 7/10
375/375 [==============================] - 27s 69ms/step - loss: 0.0754 -

2022/03/24 03:30:29 INFO mlflow.tracking.fluent: Experiment with name 'resnet50_v2' does not exist. Creating a new experiment.


MLflow Experiment ID: 2
MLflow Run ID: dcc8e59e5693465f90fa67df9c762eda
INFO:tensorflow:Assets written to: /tmp/tmpidpraigd/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpidpraigd/model/data/model/assets
2022/03/24 03:31:03 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.2+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


# efficientnetB0 model

In [14]:
from tensorflow.keras import layers
import tensorflow as tf

In [15]:
#base model
effnetb0_base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                                                           weights='imagenet',
                                                           pooling='avg')
effnetb0_base_model.trainable = False

#create model
input_ = layers.Input(shape=(224, 224, 3), name='input_layer')
x = effnetb0_base_model(input_, training=False)
output_ = layers.Dense(units=len(class_names), activation='softmax', name='output_layer')(x)
model = tf.keras.Model(inputs=[input_], outputs=[output_])

In [16]:
ktrain_obj = KTrain()
ktrain_obj.train_model(model, train_set, test_set, class_names, epochs=10, expt_name='effnetb0', run_name='baseline')

Compiling model...
Writing logs to /tmp/tmpj0oppbc7
Epoch 1/10
375/375 [==============================] - 34s 72ms/step - loss: 1.4845 - accuracy: 0.6930 - val_loss: 0.7417 - val_accuracy: 0.8078
Epoch 2/10
375/375 [==============================] - 27s 69ms/step - loss: 0.5298 - accuracy: 0.8586 - val_loss: 0.6150 - val_accuracy: 0.8164
Epoch 3/10
375/375 [==============================] - 27s 69ms/step - loss: 0.3779 - accuracy: 0.9027 - val_loss: 0.5748 - val_accuracy: 0.8273
Epoch 4/10
375/375 [==============================] - 27s 68ms/step - loss: 0.2895 - accuracy: 0.9320 - val_loss: 0.5610 - val_accuracy: 0.8320
Epoch 5/10
375/375 [==============================] - 28s 70ms/step - loss: 0.2313 - accuracy: 0.9493 - val_loss: 0.5611 - val_accuracy: 0.8242
Epoch 6/10
375/375 [==============================] - 27s 69ms/step - loss: 0.1873 - accuracy: 0.9608 - val_loss: 0.5537 - val_accuracy: 0.8297
Epoch 7/10
375/375 [==============================] - 27s 69ms/step - loss: 0.1535 -

INFO:tensorflow:Assets written to: /tmp/tmpxrfa8uxf/model/data/model/assets
2022/03/24 04:20:56 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.2+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


# efficientnetB0 model with data augmentation

In [12]:
#data augmentation layer
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
                        preprocessing.RandomFlip("horizontal"),
                        preprocessing.RandomRotation(0.2),
                        preprocessing.RandomZoom(0.2),
                        preprocessing.RandomHeight(0.2),
                        preprocessing.RandomWidth(0.2),
                        # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

#base model
effnetb0_base_model = tf.keras.applications.EfficientNetB0(include_top=False,
                                                           weights='imagenet',
                                                           pooling='avg')
effnetb0_base_model.trainable = False

#create model
input_ = layers.Input(shape=(224, 224, 3), name='input_layer')
x = data_augmentation(input_)
x = effnetb0_base_model(x, training=False)
output_ = layers.Dense(units=len(class_names), activation='softmax', name='output_layer')(x)
model2 = tf.keras.Model(inputs=[input_], outputs=[output_])

In [13]:
ktrain_obj = KTrain()
ktrain_obj.train_model(model2, train_set, test_set, class_names, epochs=10, expt_name='effnetb0', run_name='baseline_augmented')

Compiling model...
Writing logs to /tmp/tmpr7rcbpn3
Epoch 1/10
375/375 [==============================] - 36s 77ms/step - loss: 2.2977 - accuracy: 0.4756 - val_loss: 0.9127 - val_accuracy: 0.7797
Epoch 2/10
375/375 [==============================] - 28s 72ms/step - loss: 1.2749 - accuracy: 0.6612 - val_loss: 0.7132 - val_accuracy: 0.7992
Epoch 3/10
375/375 [==============================] - 29s 73ms/step - loss: 1.0665 - accuracy: 0.7039 - val_loss: 0.6600 - val_accuracy: 0.8031
Epoch 4/10
375/375 [==============================] - 28s 71ms/step - loss: 0.9705 - accuracy: 0.7253 - val_loss: 0.6186 - val_accuracy: 0.8078
Epoch 5/10
375/375 [==============================] - 28s 72ms/step - loss: 0.8798 - accuracy: 0.7493 - val_loss: 0.6043 - val_accuracy: 0.8031
Epoch 6/10
375/375 [==============================] - 28s 72ms/step - loss: 0.8169 - accuracy: 0.7643 - val_loss: 0.6098 - val_accuracy: 0.8125
Epoch 7/10
375/375 [==============================] - 29s 73ms/step - loss: 0.7553 -

INFO:tensorflow:Assets written to: /tmp/tmp5_2liti6/model/data/model/assets
2022/03/24 04:13:21 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.2+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


# efficientnetB0 model with data augmentation trained on 10 epochs -> unfreeze top 5 layers & train for 10 more epochs (Work in progress)

In [40]:
from tensorflow.keras.models import load_model

stored_model_path = PROJECT_PATH.joinpath('mlruns/1/a588546b2eb64b4db55c31d805b47aa0/artifacts/saved_model/data/model')
stored_model = load_model(stored_model_path)

In [41]:
for layer in stored_model.layers:
    print(layer.name, layer.trainable)

input_layer True
data_augmentation True
efficientnetb0 False
output_layer True


In [42]:
#make last 10 layers of effnetb0 base model trainable
effnet0_base = stored_model.layers[2]
effnet0_base.trainable = True

# #freeze last 10 layers of  base_model
# for layer in effnet0_base.layers[:-10]:
#     layer.trainable = False
# trainable_list = [-2, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17]

# for idx in trainable_list:
#     effnet0_base.layers[idx].trainable = True

In [43]:

for idx, layer in enumerate(stored_model.layers):
    if idx==2:
        for jdx, sublayer in enumerate(layer.layers):
            if jdx > 200: 
                print(sublayer.name, sublayer.trainable)
    

block6c_se_excite True
block6c_project_conv True
block6c_project_bn True
block6c_drop True
block6c_add True
block6d_expand_conv True
block6d_expand_bn True
block6d_expand_activation True
block6d_dwconv True
block6d_bn True
block6d_activation True
block6d_se_squeeze True
block6d_se_reshape True
block6d_se_reduce True
block6d_se_expand True
block6d_se_excite True
block6d_project_conv True
block6d_project_bn True
block6d_drop True
block6d_add True
block7a_expand_conv True
block7a_expand_bn True
block7a_expand_activation True
block7a_dwconv True
block7a_bn True
block7a_activation True
block7a_se_squeeze True
block7a_se_reshape True
block7a_se_reduce True
block7a_se_expand True
block7a_se_excite True
block7a_project_conv True
block7a_project_bn True
top_conv True
top_bn True
top_activation True
avg_pool True


In [ ]:
ktrain_obj = KTrain()
ktrain_obj.train_model(stored_model, train_set, test_set, class_names, epochs=10, lr=0.0001, expt_name='effnetb0', run_name='baseline_aug_10')

In [32]:
test_path = PROJECT_PATH.joinpath('mlruns/1/3389ac0d7a954457b51cce6d773b973c/artifacts/saved_model/data/model')
test_model = load_model(test_path)

In [33]:
test_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 output_layer (Dense)        (None, 120)               153720    
                                                                 
Total params: 4,203,291
Trainable params: 153,720
Non-trainable params: 4,049,571
_________________________________________________________________


In [34]:

for idx, layer in enumerate(test_model.layers):
    if idx==2:
        for jdx, sublayer in enumerate(layer.layers):
            if jdx > 200: 
                print(sublayer.name, sublayer.trainable)

block6c_se_excite False
block6c_project_conv False
block6c_project_bn False
block6c_drop False
block6c_add False
block6d_expand_conv False
block6d_expand_bn False
block6d_expand_activation False
block6d_dwconv False
block6d_bn False
block6d_activation False
block6d_se_squeeze False
block6d_se_reshape False
block6d_se_reduce False
block6d_se_expand False
block6d_se_excite False
block6d_project_conv False
block6d_project_bn False
block6d_drop False
block6d_add False
block7a_expand_conv False
block7a_expand_bn False
block7a_expand_activation False
block7a_dwconv False
block7a_bn False
block7a_activation False
block7a_se_squeeze False
block7a_se_reshape False
block7a_se_reduce False
block7a_se_expand False
block7a_se_excite False
block7a_project_conv False
block7a_project_bn False
top_conv True
top_bn False
top_activation True
avg_pool False
